El presente notebook contiene una limpieza de los datos con el fin de un posterior uso en un modelo.

In [468]:
import pandas as pd
import numpy as np
from os import path, getcwd
from typing import Dict


In [469]:
df= pd.read_excel('./data/Caso 01 - Cobranza Sector Financiero.xlsx')

In [470]:
import findspark
findspark.init()

In [471]:
from pyspark.sql import SparkSession
from IPython.core.display import HTML

display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [472]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType, DoubleType , LongType, DateType
from pyspark.sql.functions import col, desc, asc, isnan, when, count, avg, min, max

In [473]:
spark = SparkSession.builder.appName("PysparkSession")\
                            .config("spark.shuffle.sql.partitions", 5)\
                            .getOrCreate()

In [474]:
spark

In [475]:
file_name='Caso 01 - Cobranza Sector Financiero.xlsx'

In [476]:
file_base_name: str = file_name.rsplit('.', maxsplit=1)[0]

In [477]:
files_path: str = path.join(getcwd(), 'data')

In [478]:
files_path

'C:\\Users\\LENOVO\\hackathon\\data'

In [479]:
cols_schemas: Dict[str, dict] = {
    "Consecutivo Documento Deudor": {
        "new_name": "consecutivo_documento_deudor",
        "type": IntegerType(),
        "nullable": False
    },
    "Nit": {
        "new_name": "nit",
        "type": IntegerType(),
        "nullable": False
    },
    "Nombres": {
        "new_name": "nombres",
        "type": StringType(),
        "nullable": True
    },
    "Numero Documento": {
        "new_name": "numero_documento",
        "type": IntegerType(),
        "nullable": True
    },
    "Tipo De Producto": {
        "new_name": "tipo_producto",
        "type": IntegerType(),
        "nullable": True
    },
    "PRODUCTO": {
        "new_name": "producto",
        "type": StringType(),
        "nullable": False
    },
    "Valor Obligacion": {
        "new_name": "valor_obligacion",
        "type": DoubleType(),
        "nullable": False
    },
    "Valor Vencido": {
        "new_name": "valor_vencido",
        "type": DoubleType(),
        "nullable": False
    },
    "Regional": {
        "new_name": "regional",
        "type": IntegerType(),
        "nullable": False
    },
    "REGION": {
        "new_name": "region",
        "type": StringType(),
        "nullable": False
    },
    "Abogado": {
        "new_name": "abogado",
        "type": IntegerType(),
        "nullable": False
    },
    "Dias Mora": {
        "new_name": "dias_mora",
        "type": IntegerType(),
        "nullable": False
    },
    "FRANJA": {
        "new_name": "franja",
        "type": StringType(),
        "nullable": False
    },
    "Fecha Traslado Para Cobro": {
        "new_name": "fecha_traslado_cobro",
        "type": DateType(),
        "nullable": False
    },
    "Fecha Ultima Facturacion": {
        "new_name": "fecha_ultima_facturacion",
        "type": DateType(),
        "nullable": False
    },
    "Segmento": {
        "new_name": "segmento",
        "type": StringType(),
        "nullable": False
    },
    "Oficina Radicacion": {
        "new_name": "oficina_radicacion",
        "type": IntegerType(),
        "nullable": False
    },
    "Red": {
        "new_name": "red",
        "type": IntegerType(),
        "nullable": False
    },
    "Cuadrante": {
        "new_name": "cuadrante",
        "type": StringType(),
        "nullable": False
    },
    "Descripcion Causal": {
        "new_name": "descripcion_causal",
        "type": StringType(),
        "nullable": False
    },
    "Calificacion Real": {
        "new_name": "calificacion_real",
        "type": IntegerType(),
        "nullable": False
    },
    "Fecha De Perfeccionamiento": {
        "new_name": "fecha_Perfeccionamiento",
        "type": DateType(),
        "nullable": False
    },
    "Endeudamiento": {
        "new_name": "endeudamiento",
        "type": FloatType(),
        "nullable": False
    },
    "Rango endeudamiento": {
        "new_name": "rango_endeudamiento",
        "type": StringType(),
        "nullable": False
    },
    "Sector Economico": {
        "new_name": "sector_economico",
        "type": StringType(),
        "nullable": False
    },
    "Profesion": {
        "new_name": "profesion",
        "type": StringType(),
        "nullable": False
    },
    "Ocupacion": {
        "new_name": "ocupacion",
        "type": StringType(),
        "nullable": False
    },
}

In [480]:
df: pd.DataFrame = pd.read_excel(path.join(files_path, file_name), engine='openpyxl')

In [481]:
csv_path = path.join(files_path, file_base_name + ".csv")

In [482]:
df.to_csv(csv_path, index=False)

In [483]:
df.columns

Index(['Consecutivo Documento Deudor', 'Nit', 'Nombres', 'Numero Documento',
       'Tipo De Producto', 'PRODUCTO', 'Valor Obligacion', 'Valor Vencido',
       'Regional', 'REGION', 'Abogado', 'Dias Mora', 'FRANJA',
       'Fecha Traslado Para Cobro', 'Fecha Ultima Facturacion', 'Segmento',
       'Oficina Radicacion', 'Red', 'Cuadrante', 'Descripcion Causal',
       'Calificacion Real', 'Fecha De Perfeccionamiento', 'Endeudamiento',
       'Rango endeudamiento', 'Sector Economico', 'Profesion', 'Ocupacion'],
      dtype='object')

In [484]:
del df

In [485]:
def schema() -> StructType:
    return StructType([
        StructField(cols_schemas[key]['new_name'], 
                    cols_schemas[key]["type"], 
                    cols_schemas[key]["nullable"]) 
        for key in cols_schemas.keys()
    ])

In [486]:
df_spark = spark.read.format('csv')\
                     .options(header='True') \
                     .option('delimiter', ',') \
                     .schema(schema())\
                     .csv(csv_path)

In [487]:
df_spark.printSchema()

root
 |-- consecutivo_documento_deudor: integer (nullable = true)
 |-- nit: integer (nullable = true)
 |-- nombres: string (nullable = true)
 |-- numero_documento: integer (nullable = true)
 |-- tipo_producto: integer (nullable = true)
 |-- producto: string (nullable = true)
 |-- valor_obligacion: double (nullable = true)
 |-- valor_vencido: double (nullable = true)
 |-- regional: integer (nullable = true)
 |-- region: string (nullable = true)
 |-- abogado: integer (nullable = true)
 |-- dias_mora: integer (nullable = true)
 |-- franja: string (nullable = true)
 |-- fecha_traslado_cobro: date (nullable = true)
 |-- fecha_ultima_facturacion: date (nullable = true)
 |-- segmento: string (nullable = true)
 |-- oficina_radicacion: integer (nullable = true)
 |-- red: integer (nullable = true)
 |-- cuadrante: string (nullable = true)
 |-- descripcion_causal: string (nullable = true)
 |-- calificacion_real: integer (nullable = true)
 |-- fecha_Perfeccionamiento: date (nullable = true)
 |-- en

In [488]:
df_spark.show()

+----------------------------+---+-------+----------------+-------------+-------------------+----------------+-------------+--------+---------+-------+---------+------+--------------------+------------------------+-------------+------------------+---+--------------------+--------------------+-----------------+-----------------------+-------------+-------------------+--------------------+-------------+-------------+
|consecutivo_documento_deudor|nit|nombres|numero_documento|tipo_producto|           producto|valor_obligacion|valor_vencido|regional|   region|abogado|dias_mora|franja|fecha_traslado_cobro|fecha_ultima_facturacion|     segmento|oficina_radicacion|red|           cuadrante|  descripcion_causal|calificacion_real|fecha_Perfeccionamiento|endeudamiento|rango_endeudamiento|    sector_economico|    profesion|    ocupacion|
+----------------------------+---+-------+----------------+-------------+-------------------+----------------+-------------+--------+---------+-------+---------+-

In [489]:
from pyspark.sql.functions import col, desc, asc, isnan, when, count, avg, min, max

df_spark.select([
    count(when(col(column).isNull(), column)).alias(column) for column in df_spark.columns   
]).show()

+----------------------------+---+-------+----------------+-------------+--------+----------------+-------------+--------+------+-------+---------+------+--------------------+------------------------+--------+------------------+---+---------+------------------+-----------------+-----------------------+-------------+-------------------+----------------+---------+---------+
|consecutivo_documento_deudor|nit|nombres|numero_documento|tipo_producto|producto|valor_obligacion|valor_vencido|regional|region|abogado|dias_mora|franja|fecha_traslado_cobro|fecha_ultima_facturacion|segmento|oficina_radicacion|red|cuadrante|descripcion_causal|calificacion_real|fecha_Perfeccionamiento|endeudamiento|rango_endeudamiento|sector_economico|profesion|ocupacion|
+----------------------------+---+-------+----------------+-------------+--------+----------------+-------------+--------+------+-------+---------+------+--------------------+------------------------+--------+------------------+---+---------+--------

In [490]:
drop_cols = ['nit', 'nombres', 'numero_documento', 'sector_economico']

In [491]:
df_spark_new = df_spark

In [492]:
for col in drop_cols:
    df_spark_new = df_spark_new.drop(col)

In [493]:
clean_cols = ["fecha_ultima_facturacion", 'fecha_traslado_cobro', 'segmento', 'producto', 'region']

In [494]:
for col in clean_cols:
    df_spark_new = df_spark_new.na.drop(subset = [col], how = "all")

In [495]:
from pyspark.sql.functions import col, desc, asc, isnan, when, count, avg, min, max

df_spark_new.select([
    count(when(col(column).isNull(), column)).alias(column) for column in df_spark_new.columns   
]).show()

+----------------------------+-------------+--------+----------------+-------------+--------+------+-------+---------+------+--------------------+------------------------+--------+------------------+---+---------+------------------+-----------------+-----------------------+-------------+-------------------+---------+---------+
|consecutivo_documento_deudor|tipo_producto|producto|valor_obligacion|valor_vencido|regional|region|abogado|dias_mora|franja|fecha_traslado_cobro|fecha_ultima_facturacion|segmento|oficina_radicacion|red|cuadrante|descripcion_causal|calificacion_real|fecha_Perfeccionamiento|endeudamiento|rango_endeudamiento|profesion|ocupacion|
+----------------------------+-------------+--------+----------------+-------------+--------+------+-------+---------+------+--------------------+------------------------+--------+------------------+---+---------+------------------+-----------------+-----------------------+-------------+-------------------+---------+---------+
|            

In [496]:
df_spark_new.show()

+----------------------------+-------------+-------------------+----------------+-------------+--------+---------+-------+---------+------+--------------------+------------------------+-------------+------------------+---+--------------------+--------------------+-----------------+-----------------------+-------------+-------------------+-------------+-------------+
|consecutivo_documento_deudor|tipo_producto|           producto|valor_obligacion|valor_vencido|regional|   region|abogado|dias_mora|franja|fecha_traslado_cobro|fecha_ultima_facturacion|     segmento|oficina_radicacion|red|           cuadrante|  descripcion_causal|calificacion_real|fecha_Perfeccionamiento|endeudamiento|rango_endeudamiento|    profesion|    ocupacion|
+----------------------------+-------------+-------------------+----------------+-------------+--------+---------+-------+---------+------+--------------------+------------------------+-------------+------------------+---+--------------------+-------------------

In [497]:
def mode_pyspark(df_spark, list_required):
    dic_moda = {}

    list_dic = []

    for item,_ in df_spark.dtypes:

        try:
            if item in list_required:
                key , value = item+'_Moda', df_spark.filter(col(item).isNotNull()).groupBy(item)\
                                                                                  .count()\
                                                                                  .orderBy("count", ascending=False)\
                                                                                  .collect()[0]\
                                                                                  .__getitem__(item)
                dic_moda[key] = value
        except:
            pass

    return dic_moda

In [498]:
modas=mode_pyspark(df_spark_new, ['cuadrante', 'ocupacion', 'profesion', 'descripcion'])

In [499]:
modas

{'cuadrante_Moda': 'TIENE VOLUNTAD Y TIENE CAPACIDAD REDUCIDA',
 'profesion_Moda': 'SIN PROFESION',
 'ocupacion_Moda': 'SIN OFICIO'}

In [500]:
df_spark_new = df_spark_new.drop('descripcion_causal')

In [501]:
df_spark_new = df_spark_new.withColumn("cuadrante", when(df_spark_new["cuadrante"].isNull(), modas['cuadrante_Moda']).otherwise(df_spark_new["cuadrante"]))

In [502]:
df_spark_new = df_spark_new.withColumn("ocupacion", when(df_spark_new["ocupacion"].isNull(), modas['ocupacion_Moda']).otherwise(df_spark_new["ocupacion"]))

In [503]:
df_spark_new = df_spark_new.withColumn("profesion", when(df_spark_new["profesion"].isNull(), modas['profesion_Moda']).otherwise(df_spark_new["profesion"]))

In [504]:
df_spark_new.show()

+----------------------------+-------------+-------------------+----------------+-------------+--------+---------+-------+---------+------+--------------------+------------------------+-------------+------------------+---+--------------------+-----------------+-----------------------+-------------+-------------------+-------------+-------------+
|consecutivo_documento_deudor|tipo_producto|           producto|valor_obligacion|valor_vencido|regional|   region|abogado|dias_mora|franja|fecha_traslado_cobro|fecha_ultima_facturacion|     segmento|oficina_radicacion|red|           cuadrante|calificacion_real|fecha_Perfeccionamiento|endeudamiento|rango_endeudamiento|    profesion|    ocupacion|
+----------------------------+-------------+-------------------+----------------+-------------+--------+---------+-------+---------+------+--------------------+------------------------+-------------+------------------+---+--------------------+-----------------+-----------------------+-------------+-----

In [505]:
from pyspark.sql.functions import col, desc, asc, isnan, when, count, avg, min, max

df_spark_new.select([
    count(when(col(column).isNull(), column)).alias(column) for column in df_spark_new.columns   
]).show()

+----------------------------+-------------+--------+----------------+-------------+--------+------+-------+---------+------+--------------------+------------------------+--------+------------------+---+---------+-----------------+-----------------------+-------------+-------------------+---------+---------+
|consecutivo_documento_deudor|tipo_producto|producto|valor_obligacion|valor_vencido|regional|region|abogado|dias_mora|franja|fecha_traslado_cobro|fecha_ultima_facturacion|segmento|oficina_radicacion|red|cuadrante|calificacion_real|fecha_Perfeccionamiento|endeudamiento|rango_endeudamiento|profesion|ocupacion|
+----------------------------+-------------+--------+----------------+-------------+--------+------+-------+---------+------+--------------------+------------------------+--------+------------------+---+---------+-----------------+-----------------------+-------------+-------------------+---------+---------+
|                           0|            0|       0|               0|